In [31]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import re

HEADER_PATTERN = 'BatteryMgr:DataCollectionService: onStartCommand: rawFields => '
DATA_PATTERN = 'BatteryMgr:DataCollectionService: stats => '

def reset_timestamp(df):
    df['Timestamp'] = df['Timestamp'] - df['Timestamp'][0]
    df['Timestamp'] = df['Timestamp'] / 1000
    return df

def find_clumsy_directories(root='.'):
    res = []
    for dirpath, dirnames, filenames in os.walk(root):
        for dirname in dirnames:
            if 'clumsy' in dirname.lower():
                res.append(os.path.join(dirpath, dirname))
    return res

def calculate_power(df):
    df['power'] = (abs(df['BATTERY_PROPERTY_CURRENT_NOW']) / 1000 / 1000) * (df['EXTRA_VOLTAGE'] / 1000)
    return df

def trapezoid_method(df):
    return np.trapz(df['power'].values, df['Timestamp'].values), df['Timestamp'].values[-1]


def get_column_names(file):
    pattern = re.compile(HEADER_PATTERN)
    cols = []
    for line in open(file, encoding='utf-16'):
        if pattern.search(line):
            cols = line.split(HEADER_PATTERN)[1].split(',')
            cols = [col.strip() for col in cols]
            break
    return cols

def get_data(file):
    pattern = re.compile(DATA_PATTERN)
    data = []
    for line in open(file, encoding='utf-16'):
        if pattern.search(line):
            data.append(line.split(DATA_PATTERN)[1].strip('\n').split(','))

    return data

def generate_csv(data_path):
    cols = get_column_names(data_path)
    data = get_data(data_path)
    df = pd.DataFrame(data, columns=cols)
    df['Timestamp'] = np.int64(df['Timestamp'])
    df['BATTERY_PROPERTY_CURRENT_NOW'] = np.int64(df['BATTERY_PROPERTY_CURRENT_NOW'])
    df['EXTRA_VOLTAGE'] = np.int64(df['EXTRA_VOLTAGE'])
    df = reset_timestamp(df)
    df = calculate_power(df)
    df.to_csv(f'.\\{data_path.strip(".log")}.csv', index=False)
    return df

def create_dataframe(file_path):
    df = pd.DataFrame()

    df['altered_bandwidth'] = file_path.split('\\')[1].split('-')[1] == 'bandwidth'
    df['altered_dropchance'] = file_path.split('\\')[1].split('-')[1] == 'dropchance'
    df['bandwidth'] = int(file_path.split('\\')[1].split('-')[2][:-4]) if file_path.split('\\')[1].split('-')[1] == 'bandwidth' else 999999
    df['dropchance'] = float(file_path.split('\\')[1].split('-')[2]) if file_path.split('\\')[1].split('-')[1] == 'dropchance' else 0
    df['device'] = file_path.split('\\')[2]
    df['app'] = file_path.split('\\')[3].strip('-W')
    df['app_details'] = file_path.split('\\')[4]
    df['is_record'] = 'record' in file_path.split('\\')[5]
    df['is_wireless'] = '-W' in file_path.split('\\')[3]
    df['repetition'] = file_path.split('\\')[5].strip('record') if 'record' in file_path.split('\\')[5] else file_path.split('\\')[5].strip('replay')

#     print(df)
    return df

# def process_BM():
#     df_aggregated = pd.DataFrame(columns=['device', 'app', 'app_details', 'is_record', 'is_wireless', 'repetition', 'energy(J)', 'length(s)'])
#     for root, dirs, files in os.walk(".", topdown=False):
#         for name in files:
#             if name == 'battery_manager.log':
#                 # print(os.path.join(root, name))
#                 energy, length = trapezoid_method(generate_csv(os.path.join(root, name)))
#                 device = root.split('\\')[1]
#                 app = root.split('\\')[2].strip('-W')
#                 app_details = root.split('\\')[3]
#                 is_record = 'record' in root.split('\\')[4]
#                 is_wireless = '-W' in root.split('\\')[2]
#                 repetition = root.split('\\')[4].strip('record') if 'record' in root.split('\\')[4] else root.split('\\')[4].strip('replay')
#                 df_aggregated = pd.concat([df_aggregated, pd.DataFrame([[device, app, app_details, is_record, is_wireless, repetition, energy, length]], columns=['device', 'app', 'app_details', 'is_record', 'is_wireless', 'repetition', 'energy(J)', 'length(s)'])])
#     return df_aggregated

clumsy_dirs = find_clumsy_directories()

df_aggregated = pd.DataFrame()

for clumsy in clumsy_dirs:
    for root, dirs, files in os.walk(clumsy, topdown=False):
        for name in files:
            if name == 'battery_manager.log':
                try:
                    print(os.path.join(root, name))
                    energy, length = trapezoid_method(generate_csv(os.path.join(root, name)))
#                     print(create_dataframe(os.path.join(root, name)))
                    df_aggregated = pd.concat([df_aggregated, create_dataframe(os.path.join(root, name)), pd.DataFrame([[energy, length]], columns=[['energy(J)', 'length(s)']])]) 
#                     print(df_aggregated)
                except:
                    print(f'{os.path.join(root, name)} not functional')
                    
                
# print(df_aggregated)
df = df_aggregated.reset_index(drop=True)
# df

# normalized energy consumption for better comparison
df['Energy per Second (J/s)'] = df['energy(J)'] / df['length(s)']
df.to_csv('overall_energy.csv', index=False)
df

.\clumsy-bandwidth-100Mbps\Quest-2\BeatSaber\PopStars-Medium\record1\battery_manager.log
Empty DataFrame
Columns: [altered_bandwidth, altered_dropchance, bandwidth, dropchance, device, app, app_details, is_record, is_wireless, repetition]
Index: []
.\clumsy-bandwidth-100Mbps\Quest-2\BeatSaber\PopStars-Medium\replay1.0\battery_manager.log
Empty DataFrame
Columns: [altered_bandwidth, altered_dropchance, bandwidth, dropchance, device, app, app_details, is_record, is_wireless, repetition]
Index: []
.\clumsy-bandwidth-100Mbps\Quest-2\BeatSaber\PopStars-Medium\replay1.1\battery_manager.log
Empty DataFrame
Columns: [altered_bandwidth, altered_dropchance, bandwidth, dropchance, device, app, app_details, is_record, is_wireless, repetition]
Index: []
.\clumsy-bandwidth-100Mbps\Quest-2\BeatSaber\PopStars-Medium\replay1.2\battery_manager.log
Empty DataFrame
Columns: [altered_bandwidth, altered_dropchance, bandwidth, dropchance, device, app, app_details, is_record, is_wireless, repetition]
Index: [

.\clumsy-dropchance-0.001\Quest-2\BeatSaber\PopStars-Medium\replay1.1\battery_manager.log
Empty DataFrame
Columns: [altered_bandwidth, altered_dropchance, bandwidth, dropchance, device, app, app_details, is_record, is_wireless, repetition]
Index: []
.\clumsy-dropchance-0.001\Quest-2\BeatSaber\PopStars-Medium\replay1.2\battery_manager.log
Empty DataFrame
Columns: [altered_bandwidth, altered_dropchance, bandwidth, dropchance, device, app, app_details, is_record, is_wireless, repetition]
Index: []
.\clumsy-dropchance-0.001\Quest-Pro\BeatSaber\PopStars-Medium\record1\battery_manager.log
Empty DataFrame
Columns: [altered_bandwidth, altered_dropchance, bandwidth, dropchance, device, app, app_details, is_record, is_wireless, repetition]
Index: []
.\clumsy-dropchance-0.001\Quest-Pro\BeatSaber\PopStars-Medium\replay1.0\battery_manager.log
Empty DataFrame
Columns: [altered_bandwidth, altered_dropchance, bandwidth, dropchance, device, app, app_details, is_record, is_wireless, repetition]
Index: [

KeyError: 'energy(J)'